## 13.Transformer——模型构建

**学习目标**

1. 能用代码实现基于正弦余弦函数的位置编码

2. 能用代码实现位置编码的前馈网络（FFN）

3. 能用代码实现残差连接和层规范化(Add&Norm)结构

4. 能用代码实现Transformer的编码器和解码器模块

5. 能用代码构建Transformer模型

****

我们再来回顾一下上次课程讲解的Transformer模型架构：

Transformer 模型主要由编码器（Encoder）和解码器（Decoder）两部分组成，每部分都由多个相同的层（Layer）堆叠而成。标准的 Transformer 通常包括 6 层编码器和 6 层解码器。编码器负责将输入序列转化为高维表示，解码器则根据这些表示生成输出序列。

<img src="./images/transformer.jpg" style="zoom:60%;" />

（1）每个编码器都包括两个子层：多头自注意力和前馈神经网络。每个子层后都应用了残差连接和层归一化。

（2）每个解码器包括以下三个子层：掩蔽多头自注意力、编码器-解码器注意力和前馈神经网络。同样，每个子层后也应用了残差连接和层归一化，以促进梯度流动和稳定训练。

多头注意力（Multi-Head Attention）：Transformer 通过多头注意力机制，允许模型同时从不同的表示子空间捕捉信息，增强了模型的表达能力。

位置编码（Positional Encoding）：由于 Transformer 模型本身不具备捕捉序列顺序的能力，因此引入了位置编码来提供序列中每个元素的位置信息。

层归一化（Layer Normalization）和残差连接（Residual Connections）：Transformer 使用层归一化和残差连接来促进深层网络的训练，防止梯度消失或爆炸问题。


为了能够顺利构建出Transformer模型，下面我们逐一构建Transformer的几个核心组件：

- 位置编码（Positional Encoding）
- 位置编码的前馈网络（Feed Forward Network）
- 残差连接和层规范化(Add&Norm)
- Transformer编码器（Encoder）
- Transformer解码器（Decoder）

****

In [1]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import tiktoken
import torch
from torch import nn
import torch.nn.functional as F
from tools import transformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 定义超参数
vocab_size = 100256  # GPT4的词库大小
d_model = 512          # 模型维度
num_heads = 8          # 多头注意力头数
num_layers = 6         # 编码器和解码器层数
d_ffn = 2048            # 前馈神经网络隐藏层维度
dropout = 0.1          # Dropout 概率
max_len = 1000          # 最大序列长度

In [3]:
# 使用 tiktoken 分词器
tiktok_encoder = tiktoken.get_encoding("cl100k_base")
torch.manual_seed(42)  # 固定随机种子

texts = ["同志们!我踩着地雷了", "我现在开始排雷。"]

# 将文本编码为 Token IDs
tokens = [tiktok_encoder.encode(text) for text in texts]
print("Token IDs:", tokens)

# 将 Token IDs 解码回文本
decoded_texts = [tiktok_encoder.decode(token_ids) for token_ids in tokens]
print("Decoded Texts:", decoded_texts)

# 创建嵌入层
embedding_layer = nn.Embedding(vocab_size, d_model).to(device)

# 将 Token IDs 转换为张量，并填充到相同长度
token_ids = [torch.tensor(token_ids) for token_ids in tokens]
token_ids = torch.nn.utils.rnn.pad_sequence(token_ids, batch_first=True).to(device)  # 填充到相同长度
print("Token IDs:\n", token_ids)

# 获取 Token Embedding
token_embedding = embedding_layer(token_ids)  # 形状: [batch_size, seq_len, embedding_dim]
print("词嵌入向量:\n", token_embedding.shape)

Token IDs: [[42016, 78228, 80578, 0, 37046, 164, 116, 102, 84949, 222, 30590, 97565, 35287], [37046, 47551, 19000, 56386, 61056, 97565, 1811]]
Decoded Texts: ['同志们!我踩着地雷了', '我现在开始排雷。']
Token IDs:
 tensor([[42016, 78228, 80578,     0, 37046,   164,   116,   102, 84949,   222,
         30590, 97565, 35287],
        [37046, 47551, 19000, 56386, 61056, 97565,  1811,     0,     0,     0,
             0,     0,     0]], device='cuda:0')
词嵌入向量:
 torch.Size([2, 13, 512])


### 13.1  位置编码

由于Transformer模型本身不具有处理序列顺序的能力，位置编码使得模型能够理解单词在句子中的相对位置。

我们可以使用不同频率的正弦和余弦函数为序列中的每个位置生成唯一的编码。位置编码的公式如下：

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_\text{model}}}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_\text{model}}}}\right)
$$

其中：

𝑃𝐸是位置编码矩阵；

pos是词在序列中的绝对位置（从0开始）；

𝑖是维度索引（从0开始）；

$𝑑_{\text{model}}$是模型的维度大小。

对于每个维度𝑖，位置编码包含两个值：一个正弦值和一个余弦值，分别对应偶数索引和奇数索引。

位置编码的目的是给模型提供每个词在序列中的相对位置信息，这样模型就可以利用这些信息来理解词与词之间的关系。位置编码通常被添加到词嵌入（Word Embeddings）中，然后一起输入到Transformer模型中。

Transformer的位置编码选择三角函数的官方解释是：

位置编码的每个维度都对应于一个正弦曲线。波长形成一个从2π到10000·2π的几何轨迹。我们之所以选择这个函数，是因为我们假设它可以让模型很容易地通过相对位置进行学习，因为对于任何固定的偏移量k,PEpos+k都可以表示为PEpos的线性函数。

也就是说，每个维度都是波长不同的正弦波，波长范围是2π到10000·2π，选用10000这个比较大的数是因为三角函数式有周期的，在这个范围基本上，就不会出现波长一样的情况了。然后谷歌的科学家们为了让PE值的周期更长，还交替使用sin、cos来计算PE的值，就得到了最终的公式。

In [4]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000, dropout=0.1):
        """
        初始化位置编码模块。

        参数:
        - d_model: 模型的维度。
        - max_len: 最大序列长度，默认为 1000。
        - dropout: Dropout 的概率，默认为 0.1。
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化位置编码矩阵
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # 增加 batch 维度
        self.register_buffer('pe', pe)  # 将位置编码注册为缓冲区

    def forward(self, X):
        """
        前向传播。

        参数:
        - X: 输入张量，形状为 (batch_size, seq_len, d_model)。

        返回:
        - 输出张量，形状为 (batch_size, seq_len, d_model)。
        """
        X = X + self.pe[:, :X.size(1), :]  # 将位置编码加到输入上
        return self.dropout(X)

In [5]:
pos_encoder = PositionalEncoding(d_model=d_model).to(device)
# 为 Token Embedding 添加位置编码
input_embedding = pos_encoder(token_embedding)  # 形状: [batch_size, seq_len, embedding_dim]
print("输入嵌入向量:\n", input_embedding)
print("\n输入嵌入向量的形状:\n", input_embedding.shape)

输入嵌入向量:
 tensor([[[-0.0699,  1.3706,  1.4708,  ...,  0.0000, -1.6456,  0.3474],
         [-0.5197, -1.2748,  0.8772,  ...,  1.5548,  1.5875,  0.1868],
         [ 0.9168, -2.0793, -0.2925,  ...,  1.2703, -0.9609,  0.6663],
         ...,
         [ 0.0181, -0.6386, -1.3335,  ...,  0.4053, -0.3721,  0.1877],
         [-0.0000,  0.0208, -2.1964,  ...,  1.7799,  0.1963,  0.6811],
         [ 0.0543, -0.0889, -0.6966,  ...,  2.0988,  0.2045,  2.0233]],

        [[-0.0000,  1.9591,  1.3698,  ...,  0.5399, -0.3399,  2.6830],
         [ 1.8715,  0.6171,  2.3728,  ...,  0.6622,  1.8515,  0.9824],
         [ 1.1599, -0.2656,  0.0497,  ...,  0.8904,  0.0000,  0.0144],
         ...,
         [ 1.5365,  0.7202,  0.0000,  ...,  1.5813, -0.3754,  1.6866],
         [ 1.0299,  1.6575, -0.0293,  ...,  1.5813, -0.3753,  1.6866],
         [ 1.5448,  2.5902,  0.0716,  ...,  0.0000, -0.3752,  1.6866]]],
       device='cuda:0', grad_fn=<NativeDropoutBackward0>)

输入嵌入向量的形状:
 torch.Size([2, 13, 512])


In [6]:
# 实例化多头自注意力
attention = transformer.MultiHeadSelfAttention(d_model, num_heads, dropout=0.5).to(device)
attention.eval()
# 打印多头自注意力输出的形状
mul_att_shape = attention(input_embedding).shape
print(f'多头自注意力形状：{mul_att_shape}')

多头自注意力形状：torch.Size([2, 13, 512])


### 13.2  基于位置的前馈网络（FFN）

这就是一个简单的MLP，它由两个全连接层组成。这两个全连接层的输入是来自前一层的输出和来自位置编码的向量。位置编码向量是一种映射，它将位置信息编码到输入向量中，使得模型能够捕捉到输入序列中各个位置之间的关系。

In [7]:
class PositionWiseFFN(nn.Module):
    def __init__(self, d_model, d_ffn, dropout=0.1):
        super(PositionWiseFFN, self).__init__()
        self.linear1 = nn.Linear(d_model, d_ffn)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(d_ffn, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, X):
        return self.linear2(self.dropout(self.relu(self.linear1(X))))

### 13.3  残差连接和层规范化(Add&Norm)

（1）什么是层规范化？

层规范化（Layer Normalization, LayerNorm）是一种用于神经网络的归一化技术，主要用于稳定神经网络的训练过程。它与批量规范化（Batch Normalization, BatchNorm）类似，但适用的场景和实现方式有所不同。层规范化是对输入张量的**每个样本**进行归一化，而不是对整个批次进行归一化。具体来说，它对输入张量的**最后一个维度**（通常是特征维度）进行归一化，使得每个样本的特征分布更加稳定。

（2）层规范化的计算公式

对于输入张量x，层规范化的计算公式如下：

$$
\text{LayerNorm}(x) = \gamma \left( \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} \right) + \beta
$$
其中：

-  x ：输入张量，形状为  (N, *) ，其中  N  是样本数，*表示任意其他维度。

-  $\mu$：输入张量在最后一个维度上的均值，计算公式为：
  $$
  \mu = \frac{1}{H} \sum_{i=1}^H x_i
  $$
  其中  H  是最后一个维度的大小。

- $\sigma^2$ ：输入张量在最后一个维度上的方差，计算公式为：
  $$
  \sigma^2 = \frac{1}{H} \sum_{i=1}^H (x_i - \mu)^2
  $$
  

- $\epsilon$：一个很小的常数，用于防止分母为零。

- $\gamma$  和 $\beta$ ：可学习的缩放因子和偏移因子，形状与输入张量的最后一个维度相同。

（3）层规范化与批量规范化的对比

| 特性           | 批量规范化（BatchNorm）          | 层规范化（LayerNorm）                |
| -------------- | -------------------------------- | ------------------------------------ |
| **归一化维度** | 对批次维度进行归一化             | 对每个样本的最后一个维度进行归一化   |
| **适用场景**   | 适用于固定大小的批次和图像数据   | 适用于变长序列和小批次数据           |
| **计算方式**   | 依赖批次的统计信息（均值和方差） | 依赖每个样本的统计信息（均值和方差） |
| **可学习参数** | 有可学习的缩放因子和偏移因子     | 有可学习的缩放因子和偏移因子         |
| **性能**       | 在批次较大时效果好               | 在批次较小时效果好                   |

（4）层规范化的计算示例

假设输入张量  x  的形状为  (3, 4) ，表示 3 个样本，每个样本有 4 个特征。

#### 输入张量

$$
x = \begin{bmatrix}
1 & 2 & 3 & 4 \\
2 & 3 & 4 & 5 \\
3 & 4 & 5 & 6 \\
\end{bmatrix}
$$



#### 计算均值和方差

- 对每个样本计算均值和方差：

  - 样本 1：

    均值 
    $$
    \mu_1 = \frac{1+2+3+4}{4} = 2.5
    $$
    方差 
    $$
    \sigma_1^2 = \frac{(1-2.5)^2 + (2-2.5)^2 + (3-2.5)^2 + (4-2.5)^2}{4} = 1.25
    $$
     

  - 样本 2：

    均值 
    $$
    \mu_2 = \frac{2+3+4+5}{4} = 3.5
    $$
    方差 
    $$
    \sigma_2^2 = \frac{(2-3.5)^2 + (3-3.5)^2 + (4-3.5)^2 + (5-3.5)^2}{4} = 1.25
    $$
     

  - 样本 3：

    均值 
    $$
    \mu_3 = \frac{3+4+5+6}{4} = 4.5
    $$
     方差 
    $$
    sigma_3^2 = \frac{(3-4.5)^2 + (4-4.5)^2 + (5-4.5)^2 + (6-4.5)^2}{4} = 1.25
    $$
     

 层规范化的计算结果：

  $$
  \text{LayerNorm}(x) = \begin{bmatrix}
  \frac{1-2.5}{\sqrt{1.25 + \epsilon}} & \frac{2-2.5}{\sqrt{1.25 + \epsilon}} & \frac{3-2.5}{\sqrt{1.25 + \epsilon}} & \frac{4-2.5}{\sqrt{1.25 + \epsilon}} \\
  \frac{2-3.5}{\sqrt{1.25 + \epsilon}} & \frac{3-3.5}{\sqrt{1.25 + \epsilon}} & \frac{4-3.5}{\sqrt{1.25 + \epsilon}} & \frac{5-3.5}{\sqrt{1.25 + \epsilon}} \\
  \frac{3-4.5}{\sqrt{1.25 + \epsilon}} & \frac{4-4.5}{\sqrt{1.25 + \epsilon}} & \frac{5-4.5}{\sqrt{1.25 + \epsilon}} & \frac{6-4.5}{\sqrt{1.25 + \epsilon}} \\
  \end{bmatrix}
  $$
  


In [8]:
# 残差连接和层规范化(Add&Norm)
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout):
        """
        初始化 AddNorm 模块。

        参数:
        - normalized_shape: 层规范化的输入形状（通常是输入张量的最后一个维度）。
        - dropout: Dropout 的概率（用于正则化）。
        """
        super(AddNorm, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
        
    def forward(self, X, Y):
        """
        前向传播函数。

        参数:
        - X: 输入张量（通常是上一层的输出）。
        - Y: 需要与 X 进行残差连接的张量（通常是当前层的输出）。

        返回:
        - 经过残差连接和层规范化后的输出张量。
        """
        # 对 Y 应用 Dropout，然后与 X 进行残差连接，最后进行层规范化
        return self.ln(self.dropout(Y) + X)

该模块实现了 残差连接（Residual Connection） 和 层规范化（LayerNorm） 的功能。

残差连接通过将当前层的输出与上一层的输出相加，帮助模型更好地训练深层网络。

层规范化对输入进行归一化，使得模型对输入的分布更加稳定。

Dropout 用于正则化，防止模型过拟合。

### 13.4  编码器-解码器结构

1. 编码器

结合上述编写的多头自注意力模块、位置编码模块、前馈网络、残差连接等模块，可以构建编码器。

In [9]:
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ffn, dropout):
        """
        参数:
        - d_model: 模型的维度（每个词向量的维度）。
        - num_heads: 多头注意力机制中的头数。
        - d_ffn: 前馈神经网络中隐藏层的维度。
        - dropout: Dropout 的概率。
        """
        super(EncoderBlock, self).__init__()

        # 多头自注意力层
        self.attention = transformer.MultiHeadSelfAttention(d_model, num_heads, dropout)

        # 前馈神经网络层
        self.ffn = PositionWiseFFN(d_model, d_ffn, dropout)

        # 两个 LayerNorm 层，用于归一化
        self.norm1 = nn.LayerNorm(d_model)  # 用于自注意力层的输出
        self.norm2 = nn.LayerNorm(d_model)  # 用于前馈神经网络的输出

        # Dropout 层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, X, valid_lens=None):
        """
        参数:
        - X: 编码器的输入，形状为 (batch_size, src_seq_len, d_model)。
        - valid_lens: 有效序列长度，用于屏蔽填充部分，形状为 (batch_size,)。

        返回:
        - X: 编码器的输出，形状为 (batch_size, src_seq_len, d_model)。
        """
        # 1. 多头自注意力机制
        # 输入: X (源语言序列)
        # 使用 valid_lens 屏蔽填充部分
        attention_output = self.attention(X, valid_lens)  # (batch_size, src_seq_len, d_model)
        # 残差连接 + Dropout
        X = X + self.dropout(attention_output)  # (batch_size, src_seq_len, d_model)
        # 层归一化
        X = self.norm1(X)  # (batch_size, src_seq_len, d_model)

        # 2. 前馈神经网络
        # 输入: X (自注意力输出)
        ffn_output = self.ffn(X)  # (batch_size, src_seq_len, d_model)
        # 残差连接 + Dropout
        X = X + self.dropout(ffn_output)  # (batch_size, src_seq_len, d_model)
        # 层归一化
        X = self.norm2(X)  # (batch_size, src_seq_len, d_model)

        # 返回编码器的输出
        return X

In [10]:
encoder = EncoderBlock(d_model, num_heads, d_ffn, dropout).to(device)

encoder_outputs = encoder(input_embedding)
print(encoder_outputs.shape)

torch.Size([2, 13, 512])


2. 解码器

同样，我们也可以使用以上组件来构建一个transformer解码器。编码器的输出是解码器的输入。


In [11]:
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ffn, dropout):
        """
        参数:
        - d_model: 模型的维度（每个词向量的维度）。
        - num_heads: 多头注意力机制中的头数。
        - d_ffn: 前馈神经网络中隐藏层的维度。
        - dropout: Dropout 的概率。
        """
        super(DecoderBlock, self).__init__()

        # 第一个多头自注意力层（用于解码器自注意力）
        self.attention1 = transformer.MultiHeadSelfAttention(d_model, num_heads, dropout)

        # 第二个多头自注意力层（用于编码器-解码器注意力）
        self.attention2 = transformer.MultiHeadSelfAttention(d_model, num_heads, dropout)

        # 前馈神经网络层
        self.ffn = PositionWiseFFN(d_model, d_ffn, dropout)

        # 三个 LayerNorm 层，用于归一化
        self.norm1 = nn.LayerNorm(d_model)  # 用于第一个注意力层的输出
        self.norm2 = nn.LayerNorm(d_model)  # 用于第二个注意力层的输出
        self.norm3 = nn.LayerNorm(d_model)  # 用于前馈神经网络的输出

        # Dropout 层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, X, encoder_output, src_mask=None, tgt_mask=None):
        """
        参数:
        - X: 解码器的输入，形状为 (batch_size, tgt_seq_len, d_model)。
        - encoder_output: 编码器的输出，形状为 (batch_size, src_seq_len, d_model)。
        - src_mask: 源语言的掩码，用于屏蔽填充部分，形状为 (batch_size, src_seq_len)。
        - tgt_mask: 目标语言的掩码，用于屏蔽未来信息，形状为 (batch_size, tgt_seq_len)。

        返回:
        - X: 解码器的输出，形状为 (batch_size, tgt_seq_len, d_model)。
        """
        # 1. 自注意力机制（解码器自注意力）
        # 输入: X (目标语言序列)
        # 使用 tgt_mask 屏蔽未来信息
        attention_output = self.attention1(X, tgt_mask)  # (batch_size, tgt_seq_len, d_model)
        # 残差连接 + Dropout
        X = X + self.dropout(attention_output)  # (batch_size, tgt_seq_len, d_model)
        # 层归一化
        X = self.norm1(X)  # (batch_size, tgt_seq_len, d_model)

        # 2. 编码器-解码器注意力机制
        # 输入: X (解码器自注意力输出) 和 encoder_output (编码器输出)
        # 使用 src_mask 屏蔽源语言中的填充部分
        attention_output = self.attention2(X, src_mask)  # (batch_size, tgt_seq_len, d_model)
        # 残差连接 + Dropout
        X = X + self.dropout(attention_output)  # (batch_size, tgt_seq_len, d_model)
        # 层归一化
        X = self.norm2(X)  # (batch_size, tgt_seq_len, d_model)

        # 3. 前馈神经网络
        # 输入: X (编码器-解码器注意力输出)
        ffn_output = self.ffn(X)  # (batch_size, tgt_seq_len, d_model)
        # 残差连接 + Dropout
        X = X + self.dropout(ffn_output)  # (batch_size, tgt_seq_len, d_model)
        # 层归一化
        X = self.norm3(X)  # (batch_size, tgt_seq_len, d_model)

        # 返回解码器的输出
        return X

In [12]:
decoder = DecoderBlock(d_model, num_heads, d_ffn, dropout).to(device)

outputs = decoder(encoder_outputs, input_embedding)
print(outputs.shape)

torch.Size([2, 13, 512])


### 13.5 Transformer模型构建

In [13]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ffn, dropout, max_len=1000):
        """
        参数:
        - src_vocab_size: 源语言词汇表的大小。
        - tgt_vocab_size: 目标语言词汇表的大小。
        - d_model: 模型的维度（每个词向量的维度）。
        - num_heads: 多头注意力机制中的头数。
        - num_layers: 编码器和解码器的层数。
        - d_ffn: 前馈神经网络中隐藏层的维度。
        - dropout: Dropout 的概率。
        - max_len: 序列的最大长度，默认为 1000。
        """
        super(Transformer, self).__init__()

        # 编码器的词嵌入层，将源语言的词索引映射为 d_model 维的向量
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)

        # 解码器的词嵌入层，将目标语言的词索引映射为 d_model 维的向量
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        # 位置编码模块，为输入序列添加位置信息
        self.positional_encoding = PositionalEncoding(d_model, max_len)

        # 编码器层，由多个 EncoderBlock 组成
        self.encoder_layers = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ffn, dropout) for _ in range(num_layers)
        ])

        # 解码器层，由多个 DecoderBlock 组成
        self.decoder_layers = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ffn, dropout) for _ in range(num_layers)
        ])

        # 线性层，将解码器的输出映射为目标语言词汇表的大小
        self.linear = nn.Linear(d_model, tgt_vocab_size)

        # Dropout 层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        参数:
        - src: 源语言输入序列，形状为 (batch_size, src_seq_len)。
        - tgt: 目标语言输入序列，形状为 (batch_size, tgt_seq_len)。
        - src_mask: 源语言的掩码，用于屏蔽填充部分，形状为 (batch_size, src_seq_len)。
        - tgt_mask: 目标语言的掩码，用于屏蔽未来信息，形状为 (batch_size, tgt_seq_len)。

        返回:
        - output: 模型的输出，形状为 (batch_size, tgt_seq_len, tgt_vocab_size)。
        """
        # 编码器部分
        # 1. 对源语言输入进行词嵌入
        src_embedded = self.encoder_embedding(src)  # (batch_size, src_seq_len, d_model)
        # 2. 添加位置编码
        src_embedded = self.positional_encoding(src_embedded)  # (batch_size, src_seq_len, d_model)
        # 3. 应用 Dropout
        src_embedded = self.dropout(src_embedded)  # (batch_size, src_seq_len, d_model)
        # 4. 通过多个编码器层
        encoder_output = src_embedded
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output, src_mask)  # (batch_size, src_seq_len, d_model)

        # 解码器部分
        # 1. 对目标语言输入进行词嵌入
        tgt_embedded = self.decoder_embedding(tgt)  # (batch_size, tgt_seq_len, d_model)
        # 2. 添加位置编码
        tgt_embedded = self.positional_encoding(tgt_embedded)  # (batch_size, tgt_seq_len, d_model)
        # 3. 应用 Dropout
        tgt_embedded = self.dropout(tgt_embedded)  # (batch_size, tgt_seq_len, d_model)
        # 4. 通过多个解码器层
        decoder_output = tgt_embedded
        for layer in self.decoder_layers:
            decoder_output = layer(decoder_output, encoder_output, src_mask, tgt_mask)  # (batch_size, tgt_seq_len, d_model)

        # 输出:将解码器的输出映射为目标语言词汇表的大小
        output = self.linear(decoder_output)  # (batch_size, tgt_seq_len, tgt_vocab_size)
        return output

In [14]:
# 实例化 Transformer 模型
transformer = Transformer(vocab_size, vocab_size, d_model, num_heads, num_layers, d_ffn, dropout, max_len).to(device)

# 将 Token IDs 解码回文本
decoded_texts = [tiktok_encoder.decode(token_ids) for token_ids in tokens]
print("Decoded Texts:", decoded_texts)

src = token_ids[0].unsqueeze(0)  # (batch_size, src_seq_len)
tgt = token_ids[1].unsqueeze(0)  # (batch_size, tgt_seq_len)
print("源序列:\n", src)
print("目标序列:\n", tgt)

# 前向传播
output = transformer(src, tgt)
print("\nTransformer 输出形状:\n", output.shape)  # (batch_size, tgt_seq_len, tgt_vocab_size)

Decoded Texts: ['同志们!我踩着地雷了', '我现在开始排雷。']
源序列:
 tensor([[42016, 78228, 80578,     0, 37046,   164,   116,   102, 84949,   222,
         30590, 97565, 35287]], device='cuda:0')
目标序列:
 tensor([[37046, 47551, 19000, 56386, 61056, 97565,  1811,     0,     0,     0,
             0,     0,     0]], device='cuda:0')

Transformer 输出形状:
 torch.Size([1, 13, 100256])
